In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [3]:
loc = 'cuda:{}'.format(0)
checkpoint = torch.load('checkpoint_0099.pth.tar', map_location=loc)

# create new OrderedDict that does only contain module.encoder_k statedict
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if k[:17]=='module.encoder_k.':
        name = k[17:] # remove 'module.encoder_k.'
        new_state_dict[name] = v


In [10]:
device = torch.device("cuda:0")
resnet50 = models.resnet50(num_classes=128)
dim_mlp = resnet50.fc.weight.shape[1]
print(dim_mlp)
resnet50.fc = nn.Sequential(nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), resnet50.fc)
resnet50.to(device)
resnet50.load_state_dict(new_state_dict)

2048


<All keys matched successfully>

In [7]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
resnet50.fc = nn.Linear(dim_mlp, 40)  # 将输出类别数设置为40
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

In [14]:
# 在每个训练步骤中迭代数据集
for inputs, labels in dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # 前向传播
    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

NameError: name 'dataloader' is not defined

In [4]:
class myModel(nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.encoder = resnet50
    def forward(self, x):
        k = self.encoder(x)
        k = nn.functional.normalize(k, dim=1)            
        return k